# Scripts to compute functional time-series

In [1]:
import os
import gc
import sys
import random
import datetime
import importlib
import itertools
import numpy as np
from scipy import spatial
import scipy.sparse as sparse
import scipy.stats as stats
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns

from mayavi import mlab

# os.chdir('/media/sina/Windows1/Users/smansourlako/Documents/Reserach/Codes/fMRI')
os.chdir('/home/sina/Documents/Research/Codes/fMRI')

# import constants as cs
import myconstants as cs
# import importlib.util
# spec = importlib.util.spec_from_file_location('cs', '/home/sina/Documents/Research/Codes/fMRI/constants.py')
# cs = importlib.util.module_from_spec(spec)
# sys.modules['cs'] = cs
# spec.loader.exec_module(cs)
import utils
import niutils
import hcp


In [2]:
%config Completer.use_jedi = False

---

Some observations about the fMRI:

There are regions which are not properly captured in the fMRI and this poses a potential source of noise/artefact/low quality signal from some regions. One example evident in UKB data is the fMRI quality at the ventral anterior temporal lobe (vATL) which [is known to have such problems](https://www.ncbi.nlm.nih.gov/labs/pmc/articles/PMC4627358/).

We may want to compute the mean energy (non-zero frequency) of the signal at each region to use it a measure of signal quality. If we aim to do so should we also sample from somewhere outside GM (either WM or CSF) as a noise exemplar?


---

We should first load the volumetric labels:

In [46]:
# Glasser individualized volumetric atlas

# cortical_atlas = nib.load('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser.nii.gz')
cortical_atlas = nib.load('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_resampled_to_fMRI.nii.gz')


In [47]:
cortical_atlas.shape

(88, 88, 64)

In [16]:
# label names:

cortical_labels = pd.DataFrame(
    np.genfromtxt(
        '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasserColorLUT.txt',
        dtype='str'
    ),
    columns=['index', 'label_name', 'R', 'G', 'B', 'A'],
).astype(
    dtype= {
        "index": "int",
        "label_name": "str",
        "R": "int",
        "G": "int",
        "B": "int",
        "A": "int",
    }
)


In [45]:
cortical_labels

,index,label_name,R,G,B,A
0,0,???,255,255,255,255
1,1,R_V1_ROI,69,0,255,0
2,2,R_MST_ROI,49,109,132,0
3,3,R_V6_ROI,74,90,180,0
4,4,R_V2_ROI,28,56,236,0
...,...,...,...,...,...,...
356,356,L_STSva_ROI,38,36,22,0
357,357,L_TE1m_ROI,59,45,42,0
358,358,L_PI_ROI,116,48,46,0
359,359,L_a32pr_ROI,133,69,90,0


In [5]:
# subcortical atlas (Melbourne subcortical atlas S3)

subcortical_atlas = nib.load('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/native_Tian_Subcortex_S3_3T_func_space.nii.gz')


In [6]:
subcortical_atlas.shape

(88, 88, 64)

In [20]:
np.unique(subcortical_atlas.get_fdata())

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50.])

In [73]:
# Label names:

subcortical_labels = pd.DataFrame(
    ['???'] + list(np.genfromtxt(
        '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/Tian_Subcortex_S3_3T_label.txt',
        dtype='str'
    )),
    columns=['label_name',],
).astype(
    dtype= {
        "label_name": "str",
    }
)

subcortical_labels['index'] = subcortical_labels.index


In [74]:
subcortical_labels

,label_name,index
0,???,0
1,HIP-head-m-rh,1
2,HIP-head-l-rh,2
3,HIP-body-rh,3
4,HIP-tail-rh,4
5,THA-VPm-rh,5
6,THA-VPl-rh,6
7,THA-VAi-rh,7
8,THA-VAs-rh,8
9,THA-DAm-rh,9


In [106]:
# Lets load the brain mask  (for global signal)
brain_mask = nib.load('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/fMRI/rfMRI.ica/mask.nii.gz')


In [107]:
brain_mask.shape

(88, 88, 64)

In [57]:
# Lets also create a mask of all GM regions (for GM signal)
gray_matter = ((cortical_atlas.get_fdata()>0)|(subcortical_atlas.get_fdata()>0)).astype('<f4')


---

Now let's load the ICA clean fMRI:

In [58]:
# load the ica clean fMRI
clean_fmri = nib.load('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/fMRI/rfMRI.ica/filtered_func_data_clean.nii.gz')


In [59]:
clean_fmri.shape

(88, 88, 64, 490)

In [92]:
# clean_fmri.get_fdata()[0,0,0].shape
# np.sum(cortical_atlas.get_fdata()==1)
# np.mean(clean_fmri.get_fdata()[cortical_atlas.get_fdata()==1], axis=0).shape
# [x['index'] for (i,x) in cortical_labels.iterrows() if x['label_name']!='???']

# Here, we'll average fmri signal over every parcel from atlas

cortical_atlas_fmri = cortical_labels['label_name'][cortical_labels['label_name']!='???'].copy()

cortical_atlas_fmri = pd.concat(
    [
        cortical_atlas_fmri,
        pd.DataFrame(
            np.array(
                [
                    np.mean(clean_fmri.get_fdata()[cortical_atlas.get_fdata()==x['index']], axis=0)
                    for (i,x) in cortical_labels.iterrows() if x['label_name']!='???'
                ]
            ),
            index=cortical_atlas_fmri.index,
            columns=['timepoint_{}'.format(x) for x in range(clean_fmri.shape[-1])],
        )
    ],
    axis=1
)


In [93]:
cortical_atlas_fmri.head()

,label_name,timepoint_0,timepoint_1,timepoint_2,timepoint_3,timepoint_4,timepoint_5,timepoint_6,timepoint_7,timepoint_8,...,timepoint_480,timepoint_481,timepoint_482,timepoint_483,timepoint_484,timepoint_485,timepoint_486,timepoint_487,timepoint_488,timepoint_489
1,R_V1_ROI,13338.868532,13299.197336,13354.621712,13341.424267,13338.012469,13342.756635,13350.924666,13376.025324,13347.025084,...,13432.128926,13440.217523,13423.560864,13421.452763,13423.346139,13419.065431,13425.623573,13381.509182,13372.225409,13341.106812
2,R_MST_ROI,11790.035346,11774.205986,11756.303591,11789.619314,11799.796255,11769.719635,11747.285806,11712.264108,11683.374659,...,11755.742775,11789.604803,11743.888931,11786.965934,11699.396070,11751.673153,11695.279232,11720.766988,11712.180763,11780.850434
3,R_V6_ROI,14964.956336,14840.109523,14861.922526,14909.836899,14882.406516,14839.950565,14806.723292,14880.428770,14862.947695,...,14954.138406,14982.188255,14943.770774,14903.868608,14918.267193,14912.219727,14961.370310,14915.476104,14925.113622,14836.363651
4,R_V2_ROI,12723.481146,12676.823204,12735.826634,12712.632063,12700.825969,12727.860248,12726.883181,12764.172868,12765.330086,...,12777.632850,12778.447944,12775.665366,12789.557247,12763.392682,12798.339349,12776.672489,12769.067507,12751.280616,12712.560533
5,R_V3_ROI,14678.077110,14646.912765,14638.280894,14670.818286,14639.475949,14672.145855,14658.612543,14731.743344,14727.778730,...,14693.169524,14712.125274,14677.001235,14705.025526,14715.071878,14714.986198,14750.080800,14717.515384,14686.965404,14643.591077


In [115]:
# Similarly compute fMRI for subcortical regions:

subcortical_atlas_fmri = subcortical_labels['label_name'][subcortical_labels['label_name']!='???'].copy()

subcortical_atlas_fmri = pd.concat(
    [
        subcortical_atlas_fmri,
        pd.DataFrame(
            np.array(
                [
                    np.mean(clean_fmri.get_fdata()[subcortical_atlas.get_fdata()==x['index']], axis=0)
                    for (i,x) in subcortical_labels.iterrows() if x['label_name']!='???'
                ]
            ),
            index=subcortical_atlas_fmri.index,
            columns=['timepoint_{}'.format(x) for x in range(clean_fmri.shape[-1])],
        )
    ],
    axis=1
)


In [116]:
subcortical_atlas_fmri.head()

,label_name,timepoint_0,timepoint_1,timepoint_2,timepoint_3,timepoint_4,timepoint_5,timepoint_6,timepoint_7,timepoint_8,...,timepoint_480,timepoint_481,timepoint_482,timepoint_483,timepoint_484,timepoint_485,timepoint_486,timepoint_487,timepoint_488,timepoint_489
1,HIP-head-m-rh,6027.377753,5967.539898,5915.659723,5898.858878,5883.691799,5830.312270,5835.405206,5895.282572,5852.262947,...,5873.733850,5874.572697,5900.836095,5911.617011,5920.489360,5925.663854,5923.659847,5942.485612,5904.198723,5926.672393
2,HIP-head-l-rh,6567.303499,6561.623417,6564.173027,6528.177216,6547.002389,6541.506899,6515.902509,6528.215308,6524.953368,...,6575.123631,6561.169658,6562.039184,6526.590806,6554.965051,6571.330764,6530.548317,6569.184714,6514.901500,6530.171360
3,HIP-body-rh,8472.093091,8465.407832,8481.183156,8444.672605,8442.273517,8473.865677,8415.310513,8463.697867,8468.886185,...,8459.432008,8490.299717,8443.174955,8479.851216,8492.307500,8454.477685,8482.756819,8471.897907,8436.603036,8438.540886
4,HIP-tail-rh,8294.429956,8287.087097,8395.384546,8293.305957,8318.697290,8296.430933,8293.248547,8339.834265,8305.028442,...,8306.438232,8352.707678,8340.944592,8311.462048,8362.543188,8359.077124,8278.995532,8284.475806,8284.108618,8339.342517
5,THA-VPm-rh,7067.667513,7018.143359,6933.462939,6902.073535,7085.773063,6923.122575,6870.677702,6962.650391,6938.578581,...,7026.161035,7008.778288,6963.779964,6948.934424,7014.620850,7045.774935,6959.816569,6903.791699,6900.770964,6969.583708


In [121]:
# Lets also compute mean GM signal
# gray_matter = ((cortical_atlas.get_fdata()>0)|(subcortical_atlas.get_fdata()>0)).astype('<f4')

global_signal_fmri = pd.concat(
    [
        pd.DataFrame(
            [['global_signal'] + list(np.mean(clean_fmri.get_fdata()[brain_mask.get_fdata()==1], axis=0))],
            columns=['label_name'] + ['timepoint_{}'.format(x) for x in range(clean_fmri.shape[-1])],
        ),
        pd.DataFrame(
            [['mean_GM_signal'] + list(np.mean(clean_fmri.get_fdata()[gray_matter==1], axis=0))],
            columns=['label_name'] + ['timepoint_{}'.format(x) for x in range(clean_fmri.shape[-1])],
        ),
    ],
    ignore_index=True
)



In [122]:
global_signal_fmri

,label_name,timepoint_0,timepoint_1,timepoint_2,timepoint_3,timepoint_4,timepoint_5,timepoint_6,timepoint_7,timepoint_8,...,timepoint_480,timepoint_481,timepoint_482,timepoint_483,timepoint_484,timepoint_485,timepoint_486,timepoint_487,timepoint_488,timepoint_489
0,global_signal,9252.871540,9230.028716,9224.203941,9219.543504,9212.287909,9216.405952,9203.501769,9217.815225,9220.733756,...,9229.469301,9230.709472,9224.363758,9223.184475,9225.790354,9224.337225,9227.002347,9220.466861,9216.966951,9208.725371
1,mean_GM_signal,10896.626197,10865.396441,10857.699788,10848.429194,10840.470938,10845.124407,10826.608783,10847.366481,10850.897935,...,10865.057258,10870.002540,10858.969585,10854.017044,10858.093798,10856.331320,10861.570870,10849.949194,10844.400394,10831.920427


In [124]:
2

2